<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3>
<h4 align="center"><a href="mailto:echen29@uwo.ca?subject=Day 16: Image Registration Lecture"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Day 16, March 05, 2019</h4>

## Image Registration algorithms

The purpose of image registration is to align two images/volumes in a common coordinate system. It can be used for:

* image stitching: ![stitching](http://www.easypano.com/images/keyword/photo-stitch-partial-panorama.jpg) (image curtesy of http://www.easypano.com/photo-stitch-software.html),

* fusion of multi-modal images (such as CT and MR)

<img src="registration_visualization.gif" width="650"/>
(This is an animated GIF. If not animated within Jupyter Notebook, take a look at the original file instead)

In [1]:
# To use interactive plots (mouse clicks, zooming, panning) we use the nbagg back end. We want our graphs 
# to be embedded in the notebook, inline mode, this combination is defined by the magic "%matplotlib notebook".
%matplotlib notebook

import SimpleITK as sitk

%run update_path_to_download_script
from downloaddata import fetch_data as fdata
import gui

from myshow import myshow, myshow3d

Conceptually, image registration is relatively simple: identify an object that is common in both images/volume, and spatially align them. Using the image stitching example, ![stitching](http://www.easypano.com/images/keyword/photo-stitch-partial-panorama.jpg) (image curtesy of http://www.easypano.com/photo-stitch-software.html), part of the builing appears in two adjacent pictures, thus we can spatially align them and form a new image. This is the basis for generating a panorama photo from multiple/sequential pictures.

Previously we saw a trivial example of aligning maps together:
<img src="map1.png" width="550"/> <img src="map2.png" width="550"/> 

By identify [landmarks](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1LandmarkBasedTransformInitializerFilter.html) that are common in two images, we can use it as the basis to alignment them:

<img src="map12.png" width="550"/>

In this simple case, we only have to *translate* one image relative to the other.

Uisng the example of panorama photo, we can identify several important aspects image registration:
* Similarity Metric: Given two (or part of) images, how similar are they?  Can we quantify this measure?
  * What happends if we have two image modalities where the appearance of the same anatomy are different (CT vs. MRI)?
* Interpolator/Resampling: what if these images/volumes are of different spatial resolution?
* Transform: How do we align them?  Translation, Rotation plus Translation, Scaling of one image followed by rotatio and translation?
* Optimizer: Once we aligned two images, we can measure the *goodness* of the alignment using the similarity metric. Can we do better? How do we find a new transform that gives as a better alignment?

SimpleITK/ITK supports an infrastructure for image registration, where each of these four components can be adjusted according to the input images/volumes:

<img src="..\Data\images\ITKv4RegistrationComponentsDiagram.svg" style="width:700px"/><br><br>

To register two images, we typically desginate
* one as the **fixed image**: conceptually this image is the reference (not to be changed), of which the other is going to be registered into,
* the other as the **moving image**: this is the image we apply a transformation to, thus by moving it, such that it spatially aligns with the fixed image.

Summary: To perform image registration in SimpleITK
1. Creating an instance of the registration framework, select appropriate
  * optimizer,
  * similarity metric,
  * image interpolator.
2. The registration framework only supports images with sitkFloat32 and sitkFloat64 pixel types (use the SimpleITK <a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#af8c9d7cc96a299a05890e9c3db911885">Cast()</a> function if your image's pixel type is something else),

3. Successful registration is highly dependent on initialization. In general you can:
   * Use auxiliary information or user interaction to obtain an initial transformation (avoid resampling).
   * Center the images using the <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1CenteredTransformInitializerFilter.html">CenteredTransformInitializer</a>.
   * Coarsely sample the parameter space using the <a href="https://itk.org/Doxygen/html/classitk_1_1ExhaustiveOptimizerv4.html">Exhaustive Optimizer</a> to obtain one or more initial transformation estimates.
   * Manually initialize, via direct manipulation of transformation parameters and visualization or localization of corresponding points in the two images and then use the <a href="https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1LandmarkBasedTransformInitializerFilter.html">LandmarkBasedTransformInitializer</a>.

Before we look into each component of the registration framework, let us look at the excellent explaination in the PDF document, courtesy of Dr. Aaron Ward

**take a look at AIP_Day_16_Registration_explained.pdf**